<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/mapper-soilCondition"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://drive.google.com/drive/folders/1mEQBfa-tVViVWFt27XzUP4Wr19u1iuZm"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping soil condition | Display response and predictor variables

**Author:** Jenny Hanssen, Willeke A'Campo, Zander Venter

**Description:** Script to visualize the wetland polygons (responseVar) and the raster that are used to extract the terrain, climatic and biological predictors. 



## Connect to Earth Engine 

In [5]:
# The earth engine api (ee) is standard in Google Colab
import ee
import subprocess
import pandas as pd

try:
    import geemap
    print("The packages are installed and imported.")
except ImportError:
    print('The geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

The packages are installed and imported.


In [6]:
try:
    ee.Initialize() # Try to initialize earth engine
    print("Earth Engine is authenticated.")
except ee.EEException:
    ee.Authenticate() # Authenticate earth engine if initialization fails

Earth Engine is authenticated.


In [7]:
# import local module 
from gee_functions import *

## Set project standards

In [8]:
crs_analysis = ee.Projection('EPSG:25833') # crs used for calculations within this project
crs_mapping = ee.Projection('EPSG:3857') # crs used for displaying maps
crs_wgs84 = ee.Projection('EPSG:4326')

## Import Vector Data

In [9]:
# Import Admin boundaries 
no_country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Norway'))
no_province = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Norway'))

# Import mires fc
response_var = ee.FeatureCollection("projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var")

vector_list = [no_country, no_province, response_var]
vector_list[2].getInfo()['id']

'projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var'

**Reproject and Check Metadata**

In [10]:
# Reproject Data for Mapping in GEE into standard gee. system ('EPSG:3857')
crs_dst = crs_mapping
scale = 1
vector_mapping_list = []

for vector in vector_list:
    vector_projected = vector.map(lambda f: projectFeature(f, crs_dst, scale))
    vector_mapping_list.append(vector_projected)

In [11]:
# print metadata
for vector in vector_mapping_list:
    type = vector.getInfo()['type']
    id = vector.getInfo()['id']
    crs = vector.first().getInfo()['geometry']['crs']['properties']['name']
    print(f'{type}: "{id}" \tCRS: {crs}')

FeatureCollection: "FAO/GAUL/2015/level0" 	CRS: EPSG:3857
FeatureCollection: "FAO/GAUL/2015/level1" 	CRS: EPSG:3857
FeatureCollection: "projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var" 	CRS: EPSG:3857


In [12]:
# Convert the list to an object
feature_collection_dict = ee.Dictionary({
  'collections': vector_mapping_list
})

display(feature_collection_dict)

### Styling

In [13]:
# Color palette
dem_palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
virdis = ['440154','472878','3E4A89','31688E','25838E','1E9E89','35B779','6CCE59','B5DE2C','FDE725']

# style provinces
style_pr = {'color': '444444', 
            'width': 0.5, 
            'lineType': 'solid', 
            'fillColor': '00000000'
            }

# style wetland polygons
style_my ={
    'color': '#FF007F', 
    'width': 0.8, 
    'lineType': 'solid', 
    'fillColor': '00000000', 
    }

# Set visualization parameters as a dictionary
vis_params_dem = {
    'bands': ['elevation'], 
    'min': 0, 
    'max': 4000, 
    'palette': dem_palette
    }

vis_params_chm = {
    'min': 0, 
    'max': 30, 
    'palette': virdis
    }

In [14]:
# Init Map object
Map = geemap.Map(center=[60, 8], zoom=7, interact=False)
Map.add_layer_control()

# BASEMAP
Map.add_basemap('Esri.WorldGrayCanvas')

In [15]:
# WETLAND POLYGONS
Map.addLayer(vector_mapping_list[2].style(**style_my), {}, 'Response Variable: Mires')

# PROVINCEBORDER
Map.addLayer(vector_mapping_list[1].style(**style_pr), {}, "Fylker")
Map

Map(center=[60, 8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Import Raster Data
**Terrain and Elevation Data** 


In [16]:
## Kartverket elevation 10m - Fenoscandinavia
dtm_10m = ee.Image("users/rangelandee/NINA/Raster/Fenoscandia_DTM_10m").rename('elevation')

## Høydedata 1m DTM and DSM
dtm_col = ee.ImageCollection("users/vegar/dtm1/dtmcoll")
dsm_col = ee.ImageCollection("users/vegar/dom1/domcoll")

**Reproject and Check Metadata**

In [17]:
image_collection_list = [dtm_col, dsm_col]

# NOTE this function takes long time to run as it loops over all images
# to check what the min/max scale is 
metadata_dtm_col = get_imageCollection_metadata(dtm_col.limit(3))
metadata_dsm_col = get_imageCollection_metadata(dsm_col.limit(3))

In [18]:
df = pd.DataFrame(((metadata_dtm_col),(metadata_dsm_col)), columns=['n_images', 'max_scale', 'min_scale'])
new_column = ['dtm_col', 'dsm_col']
df.insert(loc=0, column='dataset', value=new_column)
display(df)

,dataset,n_images,max_scale,min_scale
0,dtm_col,3,1,1
1,dsm_col,3,1,1


**Mosaic Høydedata's 1m-resolution DTM and DSM ImageCollections**

Mosaic uses the mean pixel value in case pixels overlap. 

In [19]:
# Ensure that the correct projection parameters are used for mosaicing
# original projection of dtm/dsm 
dtm_1m_crs= dtm_col.first().projection()
dtm_1m = ee.Image(dtm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)
dsm_1m = ee.Image(dsm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)

**Calculate Canopy Height Model**

In [20]:
# Get canopy height model (CHM) as proxy for vegetation height
chm_1m = dsm_1m.subtract(dtm_1m).rename('CHM'); # CHM = DSM - DTM
chm_1m = chm_1m.setDefaultProjection(dtm_1m_crs)
chm_10m = ee.Image(resample(
    image=chm_1m, 
    method=ee.Reducer.mean(), 
    projection=dtm_10m.projection(),
    maxPixels=65536))
chm_10m = chm_10m.setDefaultProjection(dtm_10m.projection())

In [21]:
# check metadata (crs and scale)
# mosaicing sets the scale to the standard "GEE" resolution. 
terrain_image_list = [dtm_10m, dtm_1m, dsm_1m, chm_1m, chm_10m]
print_image_metadata(terrain_image_list)

Object Type: Image
Image Bands: ['elevation']
	scale: 10 m/px
	crs: EPSG:4326
	Band 1: elevation

Object Type: Image
Image Bands: ['elevation']
	scale: 1 m/px
	crs: EPSG:25833
	Band 1: elevation

Object Type: Image
Image Bands: ['elevation']
	scale: 1 m/px
	crs: EPSG:25833
	Band 1: elevation

Object Type: Image
Image Bands: ['CHM']
	scale: 1 m/px
	crs: EPSG:25833
	Band 1: CHM

Object Type: Image
Image Bands: ['CHM']
	scale: 10 m/px
	crs: EPSG:4326
	Band 1: CHM



## Map Displaying Rasters 
- NOTE that the resample CHM 10m is not displaying correctly. 

In [24]:
# Init Map object
CHM_Map = geemap.Map(center=[60, 8], zoom=7, interact=False)
CHM_Map.add_layer_control()
CHM_Map.addLayer(chm_10m, {}, 'CHM 10m', True, 0.3)
#Map.max_zoom(15)

In [25]:
# Init Map object
Map = geemap.Map(center=[60, 8], zoom=7, interact=False)
Map.add_layer_control()

# BASEMAP
Map.add_basemap('Esri.WorldShadedRelief')

# FYLKER 
Map.addLayer(vector_mapping_list[1].style(**style_pr), {}, "Fylker")

# Terrain 
# dtm 10m 
Map.addLayer(dtm_10m, vis_params_dem, 'DTM mosaic', True, 0.3)
# dtm 1m 
Map.addLayer(dtm_1m, vis_params_dem, 'DTM 1m', True, 0.3)
Map.addLayer(dsm_1m, vis_params_dem, 'DSM 1m', True, 0.3)

# CHM 
Map.addLayer(chm_1m, vis_params_chm, 'CHM 1m', True, 0.3)
Map.addLayer(chm_10m, vis_params_chm, 'CHM 10m', True, 0.3)

# WETLAND POLYGONS
Map.addLayer(vector_mapping_list[2].style(**style_my), {}, 'Response Variable: Mires')

Map

Map(center=[60, 8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…